In [1]:
!cat /proc/meminfo

MemTotal:       32949048 kB
MemFree:        30109136 kB
MemAvailable:   31748544 kB
Buffers:          660712 kB
Cached:          1180768 kB
SwapCached:            0 kB
Active:          1706788 kB
Inactive:         762344 kB
Active(anon):     627656 kB
Inactive(anon):       52 kB
Active(file):    1079132 kB
Inactive(file):   762292 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                72 kB
Writeback:             0 kB
AnonPages:        628084 kB
Mapped:           150216 kB
Shmem:                60 kB
Slab:             286020 kB
SReclaimable:     261892 kB
SUnreclaim:        24128 kB
KernelStack:        4448 kB
PageTables:         7764 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    16474524 kB
Committed_AS:    2300540 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [2]:
!pip install xgboost

    100% |████████████████████████████████| 15.8MB 3.2MB/s eta 0:00:01
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import gc

from sklearn import preprocessing as prp
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import (accuracy_score, r2_score, confusion_matrix,
f1_score, recall_score, precision_score)

import os
import boto3
import re

import xgboost as xgb

# ruta de trabajo en s3
bucket = 'boi-banregio'
target = 'BMI'
feat_part_number = 40

# s3 resource
s3_bucket_resource = boto3.resource('s3').Bucket(bucket)

In [3]:
# Aquí colocamos la ruta de nuestro directorio, el nombre de nuestro archivo y si tiene o no cabecera
prefix = 'datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION'

In [4]:
target_file_name = "TARGETS.csv"
target_file_key = "{}/{}".format(prefix, target_file_name)
target_obj = s3_bucket_resource.Object(target_file_key).get()
data_target = pd.read_csv(io.BytesIO(target_obj['Body'].read())).loc[:,[target]]

In [5]:
data_feature_list = []
for k in range(0, 10):
    data_file_name = "DATA_PART{}.csv".format(k)
    data_file_key = "{}/{}".format(prefix, data_file_name)
    print(data_file_key)
    
    feat_imp_file_name = "FEATURE_SELECTION/FEATURE_IMPORTANCE_PART{}.csv".format(k)
    feat_imp_file_key = "{}/{}".format(prefix, feat_imp_file_name)
    print(feat_imp_file_key)    
    
    print('Loading data')
    data_part_obj = s3_bucket_resource.Object(data_file_key).get()
    data_part = pd.read_csv(io.BytesIO(data_part_obj['Body'].read()))
    display(data_part)
    
    feat_imp_part_obj = s3_bucket_resource.Object(feat_imp_file_key).get()
    feat_imp_part = pd.read_csv(io.BytesIO(feat_imp_part_obj['Body'].read()))
    display(feat_imp_part)
    #break
    features = feat_imp_part.loc[:feat_part_number-1, 'Feature'].values
    display(features)
    #break
    
    data_feat_imp_part = data_part[features]
    data_feature_list.append(data_feat_imp_part) 
    print('Step: ', k)

print('Uploading raw feature importance')
model_features = pd.concat(data_feature_list, axis=1)
model_features_file_name = "MODEL_DATASET/RAW_MODEL_FEATURES.csv"
model_features_obj_key =  "{}/{}".format(prefix, model_features_file_name)    

f_str = io.StringIO()
model_features.to_csv(f_str, index=False)
s3_bucket_resource.Object(model_features_obj_key).put(Body=f_str.getvalue())
print('Model raw features saved!')

datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/DATA_PART0.csv
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/FEATURE_SELECTION/FEATURE_IMPORTANCE_PART0.csv
Loading data


,AVG_PND_MENOS_BAL_12M,AVG_PND_MAS_BAL_12M,AVG_BAL_12M,MAX_D_CRED_LIM_TO_BAL_03M,AVG_D_CRED_LIM_TO_BAL_06M,AVG_R_BAL_BY_CRED_LIM_12M,MIN_R_BAL_BY_MAX_BAL_03M,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,AVG_R_BAL_BY_MAX_BAL_12M,BK12_MAX_CREDIT_AMT,...,PCT_BK_CS_CL_ACC,PCT_BK_CS_OP_ACC,PCT_BK_LSG_OP_ACC,PCT_BRG_CS_OP_ACC,PCT_GBN_CF_CL_ACC,PCT_GBN_PQ_CL_ACC,PCT_GBN_PQ_OP_ACC,PCT_LSE_CS_OP_ACC,PCT_UN_CRE_CF_CL_ACC,PCT_UN_CRE_CL_ACC
0,2.784000e+03,2.784000e+03,2.784000e+03,147216.00,1.472160e+05,0.018560,1.000000,1.000000,1.000000,12.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0,0.0,0.0,0.0
1,2.986513e+03,3.661556e+03,3.492795e+03,145798.41,1.465072e+05,0.023285,1.000000,1.000000,1.000000,13.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0,0.0,0.0,0.0
2,3.357264e+03,4.492689e+03,4.222350e+03,145798.41,1.457777e+05,0.028149,1.000000,1.000000,1.000000,13.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0,0.0,0.0,0.0
3,3.668540e+03,3.586834e+03,3.616015e+03,148202.99,1.461066e+05,0.024107,0.316294,0.931629,0.829073,11.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0,0.0,0.0,0.0
4,2.502814e+06,2.456664e+06,2.479739e+06,481183.33,5.400732e+04,0.991896,0.991803,0.996503,0.995100,72.0,...,0.052632,0.210526,0.000000,0.0,0.052632,0.0,0,0.0,0.0,0.0
5,2.497031e+06,2.461169e+06,2.479100e+06,481183.33,5.672383e+04,0.991640,0.995920,0.994761,0.994845,72.0,...,0.052632,0.210526,0.000000,0.0,0.052632,0.0,0,0.0,0.0,0.0
6,2.491855e+06,2.468175e+06,2.480015e+06,-4137.56,5.694844e+04,0.992006,0.995920,0.992948,0.995255,68.0,...,0.055556,0.166667,0.000000,0.0,0.055556,0.0,0,0.0,0.0,0.0
7,2.486672e+06,2.472040e+06,2.479356e+06,-4137.56,5.808758e+04,0.991742,0.991669,0.992709,0.994991,68.0,...,0.055556,0.166667,0.000000,0.0,0.055556,0.0,0,0.0,0.0,0.0
8,2.483435e+06,2.476818e+06,2.480127e+06,-7073.66,5.762273e+04,0.992051,0.991669,0.993274,0.995339,68.0,...,0.055556,0.166667,0.000000,0.0,0.055556,0.0,0,0.0,0.0,0.0
9,2.428275e+06,2.296312e+06,2.362294e+06,481150.00,1.974057e+05,0.944917,0.804104,0.986187,0.974312,74.0,...,0.050000,0.150000,0.000000,0.0,0.050000,0.0,0,0.0,0.0,0.0


,Feature,Importance
0,AVG_R_BAL_BY_CRED_LIM_12M,0.037700
1,DEUDA_TOT,0.037007
2,MIN_R_BAL_BY_MAX_BAL_03M,0.028920
3,MAX_D_CRED_LIM_TO_BAL_03M,0.027226
4,AVG_D_CRED_LIM_TO_BAL_06M,0.026533
5,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,0.025801
6,AVG_R_BAL_BY_MAX_BAL_12M,0.025416
7,AVG_PND_MENOS_BAL_12M,0.024954
8,BK12_MAX_CREDIT_AMT,0.021526
9,AVG_PND_MAS_BAL_12M,0.021411


array(['AVG_R_BAL_BY_CRED_LIM_12M', 'DEUDA_TOT',
       'MIN_R_BAL_BY_MAX_BAL_03M', 'MAX_D_CRED_LIM_TO_BAL_03M',
       'AVG_D_CRED_LIM_TO_BAL_06M', 'AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M',
       'AVG_R_BAL_BY_MAX_BAL_12M', 'AVG_PND_MENOS_BAL_12M',
       'BK12_MAX_CREDIT_AMT', 'AVG_PND_MAS_BAL_12M',
       'MAX_VIGENTEC_BRG_R_OP_ACC_03M', 'AVG_PND_MENOS_VIGENTEH_06M',
       'MIN_SALINI_OP_ACC', 'AVG_VIGENTEC_OP_ACC_06M',
       'AVG_VIGENTEC_BRG_OP_ACC', 'AVG_MAX_DIAS_PARA_PAGO_U06M',
       'AVG_AVG_HI_CALCAR_L_12M', 'AVG_PCT_VENC29_12M',
       'AVG_AVG_HI_CALCAR_CEIL_12M', 'AVG_PND_MENOS_MAX_HI_CALCAR_L_03M',
       'MAX_AVG2_VP_MES_U06M', 'AVG_BAL_12M',
       'AVG_PND_MENOS_AVG_HI_CALCAR_FLOOR_06M', 'PCT_BK_CS_CL_ACC',
       'PCT_BK_CS_OP_ACC', 'AVG_AVG2_VP_MES_U06M_2',
       'MIN_SALINI_BRG_CL_ACC', 'MIN_PCT_VIGENTEH_12M',
       'AVG_AVG1_VP_PER_U06M_2', 'AVG2_VP_MES_2',
       'AVG_AVG1_VP_PER_U03M_2', 'AVG_AVG2_VP_MES_U03M_2',
       'AVG_VIGENTEC_BK_R_OP_ACC_06M', 'AVG_VIGE

Step:  0
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/DATA_PART1.csv
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/FEATURE_SELECTION/FEATURE_IMPORTANCE_PART1.csv
Loading data


...

Step:  8
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/DATA_PART9.csv
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/FEATURE_SELECTION/FEATURE_IMPORTANCE_PART9.csv
Loading data


,MIN_BAL_06M,MIN_BAL_12M,AVG_PND_MENOS_CRED_LIM_03M,AVG_CRED_LIM_06M,AVG_D_CRED_LIM_TO_BAL_03M,R_BAL_BY_CRED_LIM,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_03M,AVG_R_BAL_BY_MAX_BAL_03M,BK12_NUM_EXP_PAIDONTIME,NBK12_COMM_PCT_PLUS,...,NUM_LSE_CF_CL_ACC,NUM_LSE_CS_OP_ACC,NUM_LSE_LSG_OP_ACC,NUM_OTHER_FIN_R_CL_ACC,NUM_UN_CRE_LSG_CL_ACC,NUM_UN_CRE_R_OP_ACC,PCT_BK_OP_ACC,PCT_BRG_LSE_OTHER_CL_ACC,PCT_BRG_OTHER_CL_ACC,PCT_LSE_CF_CL_ACC
0,2784.00,2784.00,1.500000e+05,1.500000e+05,1.472160e+05,0.018560,1.000000,1.000000,NaN,2,...,0.0,0,0,0.0,0.0,0,0.000000,0.0,0.0,0.0
1,2784.00,2784.00,1.500000e+05,1.500000e+05,1.457984e+05,0.028011,1.000000,1.000000,NaN,2,...,0.0,0,0,0.0,0.0,0,0.000000,0.0,0.0,0.0
2,2784.00,2784.00,1.500000e+05,1.500000e+05,1.450585e+05,0.037876,1.000000,1.000000,NaN,2,...,0.0,0,0,0.0,0.0,0,0.000000,0.0,0.0,0.0
3,1797.01,1797.01,1.500000e+05,1.500000e+05,1.462608e+05,0.011980,0.863259,0.658147,NaN,1,...,0.0,0,0,0.0,0.0,0,0.000000,0.0,0.0,0.0
4,2018816.67,2018816.67,2.500000e+06,2.500000e+06,1.132748e+05,1.001655,0.994659,0.995821,1.00,11,...,0.0,0,0,1.0,0.0,0,0.368421,0.0,0.0,0.0
5,2018816.67,2018816.67,2.500000e+06,2.500000e+06,1.128436e+05,1.004510,0.998304,0.998140,1.00,11,...,0.0,0,0,1.0,0.0,0,0.368421,0.0,0.0,0.0
6,2018816.67,2018816.67,2.500000e+06,2.500000e+06,-1.448643e+04,1.011255,0.998424,0.998605,1.00,12,...,0.0,0,0,1.0,0.0,0,0.333333,0.0,0.0,0.0
7,2018816.67,2018816.67,2.500000e+06,2.500000e+06,-1.265592e+04,1.002829,0.997163,0.996587,1.00,12,...,0.0,0,0,1.0,0.0,0,0.333333,0.0,0.0,0.0
8,2018816.67,2018816.67,2.500000e+06,2.500000e+06,-1.482275e+04,1.005122,0.997231,0.996091,1.00,12,...,0.0,0,0,1.0,0.0,0,0.333333,0.0,0.0,0.0
9,2018850.00,2018816.67,2.500000e+06,2.500000e+06,3.570816e+05,0.808223,0.955969,0.899020,1.00,12,...,0.0,0,0,1.0,0.0,0,0.350000,0.0,0.0,0.0


,Feature,Importance
0,AVG_D_CRED_LIM_TO_BAL_03M,0.038320
1,R_BAL_BY_CRED_LIM,0.036167
2,NBK12_DEUDA_TOT,0.026029
3,MIN_BAL_06M,0.024307
4,AVG_R_BAL_BY_MAX_BAL_03M,0.023329
5,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_03M,0.022742
6,MIN_BAL_12M,0.022428
7,AVG_VIGENTEC_BRG_R_OP_ACC_03M,0.018553
8,AVG_SALINI_OP_ACC,0.017496
9,MIN_VIGENTEH_12M,0.014952


array(['AVG_D_CRED_LIM_TO_BAL_03M', 'R_BAL_BY_CRED_LIM',
       'NBK12_DEUDA_TOT', 'MIN_BAL_06M', 'AVG_R_BAL_BY_MAX_BAL_03M',
       'AVG_PND_MENOS_R_BAL_BY_MAX_BAL_03M', 'MIN_BAL_12M',
       'AVG_VIGENTEC_BRG_R_OP_ACC_03M', 'AVG_SALINI_OP_ACC',
       'MIN_VIGENTEH_12M', 'AVG_MIN_VP_PER_U12M_2',
       'AVG_MS_OP_BK_R_OP_ACC', 'MIN_SALINI_BK_OP_ACC_12M',
       'AVG_MIN_VP_PER_U06M', 'AVG_MAX_VP_MES_U06M', 'PCT_BK_OP_ACC',
       'MAX_VIGENTEC_BRG_OP_ACC_03M', 'AVG_PCT_VIGENTEH_12M',
       'MIN_SALINI_BRG_OP_ACC_12M', 'MAX_AVG1_VP_MES_U12M_2',
       'AVG_MS_OP_BRG_CS_OP_ACC', 'AVG_AVG2_VP_MES_U03M',
       'MAX_VENCIDO1MAS_12M', 'AVG_MAX_DIAS_PARA_PAGO_U03M',
       'NBK12_COMM_PCT_PLUS', 'AVG_VIGENTEC_OP_ACC', 'AVG_CRED_LIM_06M',
       'MAX_AVG1_VP_MES_U03M', 'AVG_PCT_VIGENTEC_OP_ACC',
       'AVG_PND_MENOS_CRED_LIM_03M', 'AVG_PND_MAS_MAX_HI_CALCAR_L_06M',
       'MIN_SALINI_BRG_CS_OP_ACC_12M', 'MAX_AVG2_VP_PER_U06M_2',
       'AVG_SALINI_BRG_OP_ACC', 'MAX_AVG2_VP_MES_U06M_2',
  

Step:  9
Uploading feature importance
Model features saved!


In [8]:
display(model_features.describe())
display(model_features)

Model features saved!


,AVG_R_BAL_BY_CRED_LIM_12M,DEUDA_TOT,MIN_R_BAL_BY_MAX_BAL_03M,MAX_D_CRED_LIM_TO_BAL_03M,AVG_D_CRED_LIM_TO_BAL_06M,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,AVG_R_BAL_BY_MAX_BAL_12M,AVG_PND_MENOS_BAL_12M,BK12_MAX_CREDIT_AMT,AVG_PND_MAS_BAL_12M,...,AVG_PND_MAS_MAX_HI_CALCAR_L_06M,MIN_SALINI_BRG_CS_OP_ACC_12M,MAX_AVG2_VP_PER_U06M_2,AVG_SALINI_BRG_OP_ACC,MAX_AVG2_VP_MES_U06M_2,MIN_AVG_HI_CALCAR_CEIL_12M,AVG_PND_MAS_MAX_HI_CALCAR_L_12M,MAX_MS_CL_BRG_R_CL_ACC,AVG_PND_MAS_MAX_HI_CALCAR_CEIL_12M,SUM_PAGOREALIZADO_PERIODO
count,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,...,425287.000000,4.252870e+05,4.252870e+05,425287.000000,4.252870e+05,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000
mean,0.012022,3.243717,-0.483808,0.672008,0.683300,-0.341297,-0.251894,0.483345,3.288619,0.482477,...,0.263964,3.067571e+05,2.909201e+02,0.513216,5.012855e+02,0.855755,0.223056,2.168418,1.402803,6.400931
std,23.915491,350.995352,1.100280,1.825779,1.946918,0.941598,0.768336,1.180457,492.157345,1.177261,...,0.927873,1.241058e+06,3.101545e+04,1.604958,9.149831e+04,6.743288,0.885903,16.910407,5.052671,124.400862
min,-1.789694,-0.663866,-3.601348,-10.698307,-11.800963,-4.373871,-3.104854,-0.371077,-0.967742,-0.371139,...,-1.250000,-4.991370e+05,-8.061480e+00,-0.525291,-9.101890e-02,-0.944444,-1.377633,-31.000000,-1.123769,-0.255973
25%,-0.576496,-0.386555,-0.733688,-0.230336,-0.234195,-0.648744,-0.635807,-0.250403,-0.419355,-0.253856,...,-0.363637,0.000000e+00,0.000000e+00,-0.319066,-4.634485e-02,-0.944444,-0.385737,0.000000,-0.275705,-0.224162
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.423504,0.613445,0.266312,0.769664,0.765805,0.351256,0.364193,0.749597,0.580645,0.746144,...,0.636363,0.000000e+00,1.000000e+00,0.680934,9.536552e-01,0.055556,0.614263,0.000000,0.724295,0.775838
max,7368.944220,56249.680672,0.359358,103.564446,131.391148,0.439008,0.486416,20.991531,96773.225806,14.555473,...,4.454544,2.050000e+07,6.484061e+06,34.494163,2.233703e+07,164.222222,4.188006,208.000000,70.338397,24488.298307


,AVG_R_BAL_BY_CRED_LIM_12M,DEUDA_TOT,MIN_R_BAL_BY_MAX_BAL_03M,MAX_D_CRED_LIM_TO_BAL_03M,AVG_D_CRED_LIM_TO_BAL_06M,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,AVG_R_BAL_BY_MAX_BAL_12M,AVG_PND_MENOS_BAL_12M,BK12_MAX_CREDIT_AMT,AVG_PND_MAS_BAL_12M,...,AVG_PND_MAS_MAX_HI_CALCAR_L_06M,MIN_SALINI_BRG_CS_OP_ACC_12M,MAX_AVG2_VP_PER_U06M_2,AVG_SALINI_BRG_OP_ACC,MAX_AVG2_VP_MES_U06M_2,MIN_AVG_HI_CALCAR_CEIL_12M,AVG_PND_MAS_MAX_HI_CALCAR_L_12M,MAX_MS_CL_BRG_R_CL_ACC,AVG_PND_MAS_MAX_HI_CALCAR_CEIL_12M,SUM_PAGOREALIZADO_PERIODO
0,-1.743006,-0.025210,0.359358,-0.144109,-0.105921,0.439008,0.486416,-0.369388,-0.580645,-0.369483,...,0.000000,0.0,0.000000,-0.350195,0.000000,0.000000,0.000000,12.0,0.000000,0.000000
1,-1.731119,0.016807,0.359358,-0.145578,-0.106867,0.439008,0.486416,-0.369265,-0.548387,-0.368961,...,-1.151515,0.0,0.000000,-0.350195,0.000000,0.055556,-1.262592,17.0,-1.026672,0.000000
2,-1.718884,0.016807,0.359358,-0.145578,-0.107841,0.439008,0.486416,-0.369040,-0.548387,-0.368467,...,-1.151515,0.0,0.000000,-0.350195,0.000000,0.055556,-1.262592,17.0,-1.026672,0.000000
3,-1.729053,0.058824,-2.348602,-0.143087,-0.107402,0.109949,-0.127429,-0.368852,-0.612903,-0.369006,...,-0.848484,0.0,0.000000,-0.350195,0.000000,0.055556,-1.021059,21.0,-0.817812,0.000000
4,0.705472,-0.142857,0.326892,0.201820,-0.230332,0.422180,0.468819,1.147149,1.354839,1.089671,...,1.909090,0.0,-2.741131,0.933852,1.213713,1.222222,1.708266,15.0,2.435759,0.285833
5,0.704829,-0.142857,0.343199,0.201820,-0.226706,0.413795,0.467903,1.143641,1.354839,1.092350,...,1.909090,0.0,0.000000,0.933852,1.213713,1.222222,1.708266,15.0,2.435759,-0.255973
6,0.705749,-0.117647,0.343199,-0.300884,-0.226406,0.405068,0.469374,1.140501,1.225806,1.096515,...,1.909090,0.0,0.000000,0.933852,1.213713,1.013889,1.708266,18.0,2.435759,4.833803
7,0.705086,-0.117647,0.326359,-0.300884,-0.224886,0.403919,0.468427,1.137357,1.225806,1.098814,...,1.909090,0.0,0.000000,0.933852,1.213713,1.013889,1.708266,18.0,2.435759,0.657336
8,0.705862,-0.117647,0.326359,-0.303925,-0.225506,0.406636,0.469677,1.135393,1.225806,1.101655,...,1.909090,0.0,0.000000,0.933852,1.213713,1.013889,1.708266,18.0,2.435759,-0.255973
9,0.587296,-0.092437,-0.416527,0.201785,-0.038930,0.372529,0.394162,1.101933,1.419355,0.994320,...,1.909090,0.0,0.000000,0.933852,0.073968,1.013889,1.708266,21.0,2.435759,-0.255973


In [6]:
extra_features_file_name = 'EXTRA_FEATURES/JAT_MCV_VAR_VARIABLES_DIASATRASO.csv'
extra_features_file_key = "{}/{}".format(prefix, extra_features_file_name)
extra_features_obj = s3_bucket_resource.Object(extra_features_file_key).get()
extra_features_data = pd.read_csv(io.BytesIO(extra_features_obj['Body'].read()))

In [9]:
display(model_features.shape)
display(extra_features_data.shape)
display(model_features.head())
display(extra_features_data.head())

(425287, 400)

(425287, 4)

,AVG_R_BAL_BY_CRED_LIM_12M,DEUDA_TOT,MIN_R_BAL_BY_MAX_BAL_03M,MAX_D_CRED_LIM_TO_BAL_03M,AVG_D_CRED_LIM_TO_BAL_06M,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,AVG_R_BAL_BY_MAX_BAL_12M,AVG_PND_MENOS_BAL_12M,BK12_MAX_CREDIT_AMT,AVG_PND_MAS_BAL_12M,...,AVG_PND_MAS_MAX_HI_CALCAR_L_06M,MIN_SALINI_BRG_CS_OP_ACC_12M,MAX_AVG2_VP_PER_U06M_2,AVG_SALINI_BRG_OP_ACC,MAX_AVG2_VP_MES_U06M_2,MIN_AVG_HI_CALCAR_CEIL_12M,AVG_PND_MAS_MAX_HI_CALCAR_L_12M,MAX_MS_CL_BRG_R_CL_ACC,AVG_PND_MAS_MAX_HI_CALCAR_CEIL_12M,SUM_PAGOREALIZADO_PERIODO
0,0.018560,76.0,1.000000,147216.00,147216.000000,1.000000,1.000000,2.784000e+03,12.0,2.784000e+03,...,NaN,NaN,NaN,150000.0,NaN,NaN,NaN,43.0,NaN,NaN
1,0.023285,81.0,1.000000,145798.41,146507.205000,1.000000,1.000000,2.986513e+03,13.0,3.661556e+03,...,0.190476,NaN,NaN,150000.0,NaN,1.5,0.207792,48.0,0.155844,NaN
2,0.028149,81.0,1.000000,145798.41,145777.650000,1.000000,1.000000,3.357264e+03,13.0,4.492689e+03,...,0.190476,NaN,NaN,150000.0,NaN,1.5,0.207792,48.0,0.155844,NaN
3,0.024107,86.0,0.316294,148202.99,146106.646667,0.931629,0.829073,3.668540e+03,11.0,3.586834e+03,...,0.666667,NaN,NaN,150000.0,NaN,1.5,0.597403,52.0,0.448052,NaN
4,0.991896,62.0,0.991803,481183.33,54007.322857,0.996503,0.995100,2.502814e+06,72.0,2.456664e+06,...,5.000000,NaN,0.659972,1250000.0,29.20558,2.2,5.000000,46.0,5.000000,10645.0


,DIAS_ATRASO,DPCV_CAP,DPCV_INT,DPCV_MIN
0,0,61,61,61
1,0,61,61,61
2,0,61,61,61
3,0,61,61,61
4,0,61,61,61


In [10]:
print('Uploading raw features + extra data')
model_features_ext = pd.concat([model_features, extra_features_data], axis=1)
model_features_ext_file_name = "MODEL_DATASET/RAW_MODEL_FEATURES+EXTRAS.csv"
model_features_ext_obj_key =  "{}/{}".format(prefix, model_features_ext_file_name)    

f_str = io.StringIO()
model_features_ext.to_csv(f_str, index=False)
s3_bucket_resource.Object(model_features_ext_obj_key).put(Body=f_str.getvalue())
print('Model raw features+extras saved!')

Uploading feature importance
Model features saved!
